In [1]:
# standard python modules
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydataset
import seaborn as sns
import os
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler


# my modules
import src.env as env
import src.wrangle as wr

# setting random seed to 7
np.random.seed(7)

In [2]:
df = wr.clean_zillow_2017(small=True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 642070 entries, 811599 to 1353151
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   Unnamed: 0.3  642070 non-null  int64
 1   Unnamed: 0.2  642070 non-null  int64
 2   Unnamed: 0.1  642070 non-null  int64
 3   Unnamed: 0    642070 non-null  int64
 4   bedrooms      642070 non-null  int64
 5   baths         642070 non-null  int64
 6   sq_feet       642070 non-null  int64
 7   tax_value     642070 non-null  int64
 8   year_built    642070 non-null  int64
 9   tax_amount    642070 non-null  int64
 10  fips          642070 non-null  int64
dtypes: int64(11)
memory usage: 58.8 MB


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2140235 entries, 0 to 2140234
Data columns (total 9 columns):
 #   Column        Dtype
---  ------        -----
 0   Unnamed: 0.1  int64
 1   Unnamed: 0    int64
 2   bedrooms      int64
 3   baths         int64
 4   sq_feet       int64
 5   tax_value     int64
 6   year_built    int64
 7   tax_amount    int64
 8   fips          int64
dtypes: int64(9)
memory usage: 147.0 MB


In [28]:
def rename_zillow_columns(df):
    df = df.rename(columns={'bedroomcnt':'bedrooms', 
                            'bathroomcnt':'bathrooms', 
                            'calculatedfinishedsquarefeet':'sq_feet', 
                            'taxvaluedollarcnt':'tax_value',
                            'yearbuilt':'year_built',
                            'taxamount':'tax_amount'})
    return df

In [29]:
def zillow_columns_to_int(df):

    # renames columns to be more intelligable and able to be referenced
    df['bedrooms'] = df['bedrooms'].astype(int) 
    df['baths'] = df['bathrooms'].astype(int)
    df['sq_feet'] = df['sq_feet'].astype(int)
    df['tax_value'] = df['tax_value'].astype(int)
    df['year_built'] = df['year_built'].astype(int)
    df['tax_amount'] = df['tax_amount'].astype(int)
    df['fips'] = df['fips'].astype(int)
    
    # casts all column elements as integers
    #df[(list(df.columns))].astype(int) 
    
    
    return df

In [30]:
def deal_with_nulls(df):

    # fills whitespace will Naans
    df = df.replace(r'^\s*s', np.NaN, regex=True)

    # the columns which we want to drop naan values from
    naan_drop_columns = ['sq_feet', 'tax_value', 'year_built', 'tax_amount']    
    

    # drop naans based on the columns identified above
    df = df.dropna(subset = naan_drop_columns)

    return df

In [31]:
df = rename_zillow_columns(df)
df = deal_with_nulls(df)
df = zillow_columns_to_int(df)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 8 columns):
 #   Column      Dtype  
---  ------      -----  
 0   bedrooms    int64  
 1   bathrooms   float64
 2   sq_feet     int64  
 3   tax_value   int64  
 4   year_built  int64  
 5   tax_amount  int64  
 6   fips        int64  
 7   baths       int64  
dtypes: float64(1), int64(7)
memory usage: 147.0 MB


In [20]:
df = rework_zillow_columns(df)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [9]:
def wrangled_zillow_2017():
    '''
    This is a very large dataset and the values can get very wide so we will handle null values by dropping them.
    The process will be to first turn whitespace into null values and then drop rows with null values from columns 
    which we find to be important.  
    '''


    # checks to see if wrangled zillow data exists already
    # if it does, then fills df with the stored data
    if os.path.isfile('data/wrangled_zillow_2017.csv'):

        df = pd.read_csv('data/wrangled_zillow_2017.csv')


    else:
        df = wr.acquire_zillow_2017()

    # fills whitespace will Naans
    df = df.replace(r'^\s*s', np.NaN, regex=True)
  
    '''
    # renames columns to be more intelligable and able to be referenced
    df.columns = ['drop', 'bedrooms', 'bathrooms', 'sq_feet', 'tax_value', 'year_built', 'tax_amount', 'fips', 'property_type']
    
    # the columns which we want to drop naan values from
    naan_drop_columns = ['sq_feet', 'tax_value', 'year_built', 'tax_amount']    
    
    # removes old reference column
    df.drop(columns=['drop'], inplace = True)

    # drop naans based on the columns identified above
    df = df.dropna(subset = naan_drop_columns)

    # casts all column elements as integers
    df[(list(df.columns))].astype(int) 

    '''
    # dataframe which is a representative sample of 30% of the original dataframe
    df30 = df.sample(frac = .30)
    

    # Cache data
    df.to_csv('data/wrangled_zillow_2017.csv')



    return df, df30 

In [10]:
df, df30 = wrangled_zillow_2017()

In [12]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,propertylandusetypeid
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0,261.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0,261.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0,261.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0,261.0
